<a href="https://colab.research.google.com/github/manvendra7/END2.0-Assignment3-Pytorch101/blob/main/Assignment_3_Pytorch_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Importing all the packages
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
import torch.optim as optim
import random
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# extending the Dataset class of Pytorch to build our own sample dataset
class rand_num(torch.utils.data.Dataset):
  def __init__(self):
    # Generating random numbers using torch.randint between 0,9
    self.data = torch.randint(0,9,(60000,),requires_grad=True,dtype=torch.float32)

  def __getitem__(self, index):
    r = self.data[index]
    return r
  
  def __len__(self):
    return len(self.data)

In [ ]:
## Assigning data to variables
df1 = rand_num()

In [ ]:
## Downloading MNIST dataset
train_set = torchvision.datasets.MNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
## A class to combine both data -- Final data
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, images, num, labels=None, transforms=None):
        self.X = images
        self.X2 = num

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        data = self.X[i][0]
        data2 = self.X2[i]
        Y = self.X[i][1]
        Y2 = self.X2[i] + self.X[i][1]

        return data,data2,Y, Y2

In [ ]:
## Final data set 
data_combined = ImageDataset(train_set,df1)

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out1 = nn.Linear(in_features=60, out_features=10)

        self.fc3 = nn.Linear(2,2)
        self.fc4 = nn.Linear(3, 2)
        self.out2 = nn.Linear(2, 1)

    
    def forward(self, t,tab):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out1(t)
        t1 = F.softmax(t, dim=1)
        
        inp1 = torch.argmax(t1,dim=1)
        x = torch.stack((inp1,tab),dim=1).type(torch.float32) #nx2
        x = self.fc3(x)
        x = F.relu(x)
        # x = self.fc4(x)
        # x = F.relu(x)
        t2 = self.out2(x)#2x1
        return t1, t2

In [ ]:
## Checking model architecture
model = Network()
print(model)

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out1): Linear(in_features=60, out_features=10, bias=True)
  (fc3): Linear(in_features=2, out_features=2, bias=True)
  (fc4): Linear(in_features=3, out_features=2, bias=True)
  (out2): Linear(in_features=2, out_features=1, bias=True)
)


In [ ]:
# A function to check correct items predicted
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
# Running the loop and checking the loss for each epoch

## Loss for regression
#L2 loss is also known as mean squared error (MSE). L2 loss, as one can easily infer, is the mean of the squared 
#differences between the predicted variable and the label variable.

## Loss for classification
#Cross-entropy

torch.set_grad_enabled(True)
network = Network()

if torch.cuda.is_available():
    network.cuda()

train_loader = torch.utils.data.DataLoader(data_combined, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.001)

for epoch in range(20):

    total_loss_1 = 0
    total_loss_2 = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, data, y, y_sum = batch 

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        images, data, y, y_sum = images.to(device), data.to(device),y.to(device), y_sum.to(device)

        preds = network(images,data) # Pass Batch
        b = nn.MSELoss()
        a = nn.CrossEntropyLoss()

        loss_a = a(preds[0],y)
        loss_b = b(preds[1], y_sum)
        loss = loss_a + loss_b

        optimizer.zero_grad()
        loss_a.backward() # Calculate Gradients
        loss_b.backward()
        optimizer.step() # Update Weights

        total_loss_1 += loss_a.item()
        total_loss_2 += loss_b.item()
        total_correct += get_num_correct(preds[0], y)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss_image:", total_loss_1,
        "loss_sum:", total_loss_2
    )

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1 total_correct: 51877 loss_image: 958.5675059556961 loss_sum: 14483.214212417603
epoch 2 total_correct: 56177 loss_image: 915.916613817215 loss_sum: 13587.106803894043
epoch 3 total_correct: 58093 loss_image: 896.2864525318146 loss_sum: 12728.925603866577
epoch 4 total_correct: 58449 loss_image: 892.766309261322 loss_sum: 11778.961261749268
epoch 5 total_correct: 58578 loss_image: 891.2278860807419 loss_sum: 10736.875761985779
epoch 6 total_correct: 58769 loss_image: 889.3549799919128 loss_sum: 9819.636379241943
epoch 7 total_correct: 58806 loss_image: 888.9140368700027 loss_sum: 9287.045938491821
epoch 8 total_correct: 58974 loss_image: 887.2651942968369 loss_sum: 9105.999529838562
epoch 9 total_correct: 59082 loss_image: 886.0920633077621 loss_sum: 9069.842492103577
epoch 10 total_correct: 59058 loss_image: 886.2818154096603 loss_sum: 9064.952618598938
epoch 11 total_correct: 59185 loss_image: 885.17265021801 loss_sum: 9064.451206207275
epoch 12 total_correct: 59220 loss_image